In [3]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd  

In [4]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

def distance_meters(mercator_start, mercator_finish):
    return mercator_start.distance(mercator_finish)


In [5]:
def aquire_bicimad():
    df4 = pd.read_json("../data/bicimad_stations.json")
    lat = [float(i["geometry_coordinates"].split(",")[0].replace("[", "")) for index, i in df4.iterrows()]
    lon = [float(row["geometry_coordinates"].split(",")[1].replace("]", "")) for index, row in df4.iterrows()]
    df4["Latitud"] = lon
    df4["Longitud"] = lat
    newdf = df4.drop(["activate","no_available","total_bases","dock_bikes","free_bases","reservations_count","geometry_type","geometry_coordinates","light","number","id"], axis='columns')
    newdf['Distance'] = newdf.apply(lambda x: to_mercator(x['Latitud'], x['Longitud']), axis =1)
    return newdf

In [6]:
%%time
def aquire_aparcamientos():
    df = pd.read_csv("../data/202625-0-aparcamientos-publicos.csv", delimiter=";")
    df1 = pd.read_csv("../data/202584-0-aparcamientos-residentes.csv", delimiter=";")
    df_concatenado = pd.concat ([df1, df] ,ignore_index=True)
    newdf = df_concatenado.drop(["PK","DESCRIPCION-ENTIDAD","HORARIO","EQUIPAMIENTO","TRANSPORTE","CONTENT-URL","NOMBRE-VIA","COORDENADA-X","COORDENADA-Y","TELEFONO","FAX","EMAIL","TIPO","Unnamed: 30","ACCESIBILIDAD","CLASE-VIAL","TIPO-NUM","NUM","PLANTA","PUERTA","ESCALERAS","ORIENTACION"], axis='columns')
    newdf.rename(columns={"NOMBRE":"Place of interest","DESCRIPCION":"Type of place",},inplace=True)
    newdf["Place address"]=newdf["LOCALIDAD"] + ", " + newdf["DISTRITO"]+ ", " + newdf["BARRIO"]+ ", " + newdf["CODIGO-POSTAL"].astype(str)
    df = newdf.drop(["LOCALIDAD","PROVINCIA","CODIGO-POSTAL","BARRIO","DISTRITO"], axis='columns')
    df_orden = df[["Place of interest", "Type of place", "Place address","LATITUD","LONGITUD"]]
    df_final = df_orden.drop([ "Type of place"], axis='columns')
    df_final['Distance'] = df_final.apply(lambda x: to_mercator(x['LATITUD'], x['LONGITUD']), axis =1)
    return df_final
                        
                        

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [7]:
aquire_aparcamientos()

,Place of interest,Place address,LATITUD,LONGITUD,Distance
0,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",40.426688,-3.699617,POINT (4500278.311459402 -412126.004286175)
1,Aparcamiento mixto. Avenida de América (interc...,"MADRID, CHAMARTIN, PROSPERIDAD, 28002.0",40.438861,-3.676622,POINT (4501633.395995631 -409560.8297186782)
2,Aparcamiento mixto. Avenida de Portugal,"MADRID, LATINA, PUERTA DEL ANGEL, 28011.0",40.411180,-3.737832,POINT (4498551.982400604 -416389.000629499)
3,Aparcamiento mixto. Brasil,"MADRID, TETUAN, CUATRO CAMINOS, 28020.0",40.456108,-3.694132,POINT (4503553.380961536 -411514.0661487146)
4,Aparcamiento mixto. Casino de la Reina,"MADRID, CENTRO, EMBAJADORES, 28005.0",40.406793,-3.704609,POINT (4498063.582233449 -412682.7875456664)
...,...,...,...,...,...
329,Aparcamiento público. Plaza del Rey,"MADRID, CENTRO, JUSTICIA, 28004.0",40.420188,-3.695872,POINT (4499554.741630848 -411708.2364780931)
330,Aparcamiento público. Santa Ana,"MADRID, CENTRO, CORTES, 28012.0",40.414878,-3.700898,POINT (4498963.673501731 -412268.8216111011)
331,Aparcamiento público. Usera,"MADRID, USERA, MOSCARDO, 28026.0",40.387103,-3.706526,POINT (4495871.690083981 -412896.6234051568)
332,Aparcamiento público. Velázquez - Juan Bravo,"MADRID, SALAMANCA, CASTELLANA, 28006.0",40.431992,-3.683450,POINT (4500868.741974963 -410322.498363965)


In [5]:
def aquire_combinado():
    df_combinado = aquire_aparcamientos().merge(aquire_bicimad(), how="cross")
    df_combinado['Distancia'] = df_combinado.apply(lambda x: distance_meters(x['Distance_x'], x['Distance_y']), axis =1)
    df_clean = df_combinado.drop(["Latitud","Longitud","Distance_x","LATITUD","LONGITUD","Distance_y"], axis='columns')
    return df_clean
    

In [6]:
def comprobar():
    b = str(input("introduce el lugar de interes: "))
    a = aquire_combinado()[aquire_combinado()["Place of interest"]==b]
    return a.sort_values(by="Distancia",ascending=True).head(1)

In [49]:
comprobar()

introduce el lugar de interes: Aparcamiento disuasorio Aviación Española


,Place of interest,Place address,name,address,Distancia
72298,Aparcamiento disuasorio Aviación Española,"MADRID, LATINA, LAS AGUILAS, 28024.0",Caramuel,Calle Caramuel nº 51,6779.868246


In [7]:
lista  = [y["Place of interest"]for i , y in aquire_combinado().iterrows()]

In [8]:
Place_of_interest = set(lista)

In [9]:
Place_of_interest1 = list(Place_of_interest)

In [29]:
aquire_combinado()

,Place of interest,Place address,name,address,Distancia
0,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",Puerta del Sol A,Puerta del Sol nº 1,1083.264542
1,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",Puerta del Sol B,Puerta del Sol nº 1,1066.886564
2,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",Miguel Moya,Calle Miguel Moya nº 1,971.121362
3,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598
4,Aparcamiento mixto. Arquitecto Ribera,"MADRID, CENTRO, JUSTICIA, 28004.0",Malasaña,Calle Manuela Malasaña nº 5,390.966753
...,...,...,...,...,...
88171,Aparcamiento público. Villa de París,"MADRID, CENTRO, JUSTICIA, 28004.0",Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4375.089652
88172,Aparcamiento público. Villa de París,"MADRID, CENTRO, JUSTICIA, 28004.0",Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,4352.063136
88173,Aparcamiento público. Villa de París,"MADRID, CENTRO, JUSTICIA, 28004.0",Facultad Biología,Calle José Antonio Novais frente al nº 12,4664.398637
88174,Aparcamiento público. Villa de París,"MADRID, CENTRO, JUSTICIA, 28004.0",Facultad Derecho,Avenida Complutense nº 23,5028.105224


In [47]:
def bicimad():
    return aquire_combinado().sort_values(by = "Distancia", ascending = True).groupby('Place of interest')['Place address', 'name','address'].nth(0)

/Users/mohamedelmarraki/miniconda3/envs/project_m1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Place address,name,address
Place of interest,,,
Aparcamiento disuasorio Aviación Española,"MADRID, LATINA, LAS AGUILAS, 28024.0",Caramuel,Calle Caramuel nº 51
Aparcamiento disuasorio Estadio Metropolitano Sur ES-02b,"MADRID, SAN BLAS-CANILLEJAS, ROSAS, 28022.0",Gutierre de Cetina,Calle Gutierre de Cetina nº 77
Aparcamiento disuasorio Fuente de la Mora,"MADRID, HORTALEZA, VALDEFUENTES, 28050.0",Manuel Caldeiro,Paseo de la Castellana nº 298
Aparcamiento disuasorio Islazul,"MADRID, CARABANCHEL, BUENAVISTA, 28054.0",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
Aparcamiento disuasorio Pitis,"MADRID, FUENCARRAL-EL PARDO, FUENTELARREINA, 2...",Pedro Rico,Calle Pedro Rico nº 4
...,...,...,...
Aparcamiento público. Plaza del Rey,"MADRID, CENTRO, JUSTICIA, 28004.0",Banco de España A,Calle Alcalá nº 49
Aparcamiento público. Santa Ana,"MADRID, CENTRO, CORTES, 28012.0",Plaza de Santa Ana,Plaza de Santa Ana nº 10
Aparcamiento público. Usera,"MADRID, USERA, MOSCARDO, 28026.0",Antonio López,Calle Eugenio Caxes frente al nº 12


In [ ]:
sum = 0
lista = []
for i in range(0, len(Place_of_interest1)):
    a = aquire_combinado()[aquire_combinado()["Place of interest"]==Place_of_interest1[i]]
    lista.append(a.sort_values(by="Distancia",ascending=True).head(1))